# 07 Semantic Kernel | 01 Plugin

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

This notebooks highlights how Plugins work. 

## Step 1: Create Semantic Kernel

MS Semantic Kernel is a tool or library developed by Microsoft to abstract away communication with LLMs and other services like vector DBs. The nuget package Microsoft.SemanticKernel provides developers with access to this powerful tool, allowing them to integrate advanced language processing capabilities into their applications effortlessly.


In [6]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: DotNetEnv, 2.5.0"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI; 
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using Kernel = Microsoft.SemanticKernel.Kernel;
using DotNetEnv;
using System.IO;

//configuration file is created during environment creation
static string _configurationFile = @"../Configuration/application.env";
Env.Load(_configurationFile);


string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "assets");
string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI1106_APIKEY") ?? "WS_AOAI1106_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI1106_ENDPOINT") ?? "WS_AOAI1106_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_AOAI1106_DEPLOYMENTNAME") ?? "WS_AOAI1106_DEPLOYMENTNAME not found";


var kernel = Kernel
    .CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        chatCompletionDeploymentName,
        oAiEndpoint,
        oAiApiKey,
        serviceId: chatCompletionDeploymentName 
    )
    .Build();



Console.WriteLine($"Semantic Kernel created...");


Installed Packages DotNetEnv, 2.5.0 Microsoft.SemanticKernel, 1.0.1

Semantic Kernel created...


## Step 2: Define Plugin

The code cell defines a so called plugin. A plugin is functionality (both native or semantic) which can be executed by Semantic Kernel. In this sample a summarization plugin is defined.



In [9]:
string skPrompt = """
    {{$ebook}}

    Summarize the content above.
    """;

OpenAIPromptExecutionSettings executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.2,
    TopP = 0.5
};

PromptTemplateConfig promptTemplateConfig = new PromptTemplateConfig(skPrompt);

KernelPromptTemplateFactory promptTemplateFactory = new KernelPromptTemplateFactory();
IPromptTemplate promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);
KernelFunction summaryFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Console.WriteLine($"Kernel plugin defined...")





Kernel plugin defined...


## Step 3: Execute Plugin

The Semantic Kernel function `InvokeAsync()` executes the defined plugin and therefore summarizes the provided ebook.

In [8]:
string ebookFile = Path.Combine(assetsFolder, "Embedding", "TheUntouchableAdolescents_Part01.txt");

string bookPartToSummarize = await File.ReadAllTextAsync(ebookFile);
var summaryResult = await kernel.InvokeAsync(summaryFunction, new() { ["ebook"] = bookPartToSummarize });

Console.WriteLine(summaryResult);


The story is about a spaceship crew that lands on the planet Diamore to warn the inhabitants about an impending disaster. The crew discovers that the Diamoraii are telepathic and resistant to accepting help due to past experiences with another alien race. Despite the crew's efforts to save them, a series of mysterious deaths occur, leading to frustration and despair. The crew's captain is determined to make one last attempt to persuade the Diamoraii to accept their help.
